<a href="https://colab.research.google.com/github/StefanATudose/NLP-Hackathon-2023/blob/main/Bert_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Versiune noua
Preprocesara facuta mai devreme(twitter + functie de normalizare fara stem)
Transfer Learning

#Setup

In [1]:
!pip install nltk
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/NitroNLP/Hackaton")
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:2"
import math
import time
import pandas as pd
import nltk
from transformers import AutoModelForSequenceClassification
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
import re
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer(language='romanian') # This one has support for Romanian too!
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import torch # Neural Networks
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel, AdamW # Transformers
from sklearn.manifold import TSNE # Data projection
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm # Progress bar
import seaborn as sns
import os
import random
from IPython import display
sns.set_style("darkgrid")
os.environ["TOKENIZERS_PARALLELISM"] = "false"
%matplotlib inline



Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
seed = 19


import numpy as np
np.random.seed(seed)
np.random.RandomState(seed)

import random
random.seed(seed)

import torch
torch.manual_seed(seed)
torch.use_deterministic_algorithms(True)

import tensorflow as tf
tf.random.set_seed(seed)

import jax
#jax.random.PRNGKey(seed) n a compilat


#Definire clase

In [4]:
def normalize_tokens(tokens):

    tokFinal = []
    for token in tokens:
        if token in string.punctuation:
            continue
        token = re.sub('ă', 'a', token)
        token = re.sub('â', 'a', token)
        token = re.sub('î', 'i', token)
        token = re.sub('ș', 's', token)
        token = re.sub('ț', 't', token)
        token = re.sub('\.', '', token)
        if token in stopwords.words("romanian"):
            continue
        if token == "":
            continue
        token = stemmer.stem(token)
        tokFinal.append(token)
    # tokFinal = [t for t in tokFinal if t not in string.punctuation]
    # tokFinal = [t for t in tokFinal if t not in stopwords.words("romanian")]
    # tokFinal = [stemmer.stem(t) for t in tokFinal]
    return tokFinal

In [5]:
# First, define the dataset
class DatasetTransformer(Dataset):
    def __init__(self, X, y, tokenizer):
        self.tokenizer = tokenizer
        self.X = X
        yList = []
        for a in y:
            if a == "non-offensive":
                yList.append(1)
            elif a == "offensive":
                yList.append(2)
            elif a == "descriptive":
                yList.append(3)
            elif a == "direct":
                yList.append(4)
            elif a == "reporting":
                yList.append(0)
        self.y = yList

            
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        text = self.X[idx]
        label = self.y[idx]
        
        # Remove cedilla diacritics as suggested in
        # https://huggingface.co/dumitrescustefan/bert-base-romanian-uncased-v1
        text = text.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș")
        text = " ".join(normalize_tokens(tweet_tokenizer.tokenize(text)))

        # Tokenize text and return the tensor
        # Maximum length set at 64 tokens for efficiency reasons. The maximum sequence length for BERT is 512.
        text_tensor = self.tokenizer.encode(text, add_special_tokens=True, max_length=64, padding='max_length', return_tensors='pt', truncation=True)

        return text_tensor, torch.tensor(label)



In [6]:
class TransformerModel(nn.Module):
    def __init__(self, in_dim=768, no_classes=5):
        super(TransformerModel, self).__init__()
        # Get the romanian Transformer from the huggingface server
        self.transformer = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-uncased-v1")
        # Add a linear layer for classification
        self.fc1 = nn.Linear(in_dim, no_classes)
        #model = BertForMultilabelSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to('cuda')
        
    def forward(self, x):
        out = x.squeeze(1)
        # Get output from Transformer.
        # We want the special [CLS] representation ([:,0,:]) from the last layer ([0]) 
        out = self.transformer(out)[0][:,0,:]
        # We usually add dropout before the final classification layer when using a Transformer
        out = F.dropout(out, p=0.1)
        out = self.fc1(out)
        return out

In [7]:
# We can find the training and testing data at ./LaRoSeDa/data_splitted
def train_test_valid(train_path='./train_data.csv', test_path='./test_data.csv', remove_punctuation=True, seed=19):
    
    # We first load the train and test data
    laroseda_train = pd.read_csv(train_path)
    laroseda_test = pd.read_csv(test_path)

    X_train = []
    y_train = []
    X_test = []
    y_test = []

    # Then we move them to some vectors in memory
    for line in range(len(laroseda_train)):
        text = laroseda_train.loc[line, "Text"]
        
        if remove_punctuation:
            text = re.sub(r'[^\w\s]', '', text)
        
        X_train.append(text)
        y_train.append(laroseda_train.loc[line, "Final Labels"])
        
    for line in range(len(laroseda_test)):
        text = laroseda_test.loc[line, "Text"]
        
        if remove_punctuation:
            text = re.sub(r'[^\w\s]', '', text)
        
        X_test.append(text)
        #y_test.append(laroseda_test.loc[line, "Final Labels"])
        
        
    # And we finally split the training data into train/valid
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=seed)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [8]:

# The tokenizer is the WordPiece tokenizer: https://towardsdatascience.com/wordpiece-subword-based-tokenization-algorithm-1fbd14394ed7
tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-uncased-v1", do_lower_case=True)


In [9]:
# Get the dataset splits once again, this time containing punctuation
X_train, X_val, X_test, y_train, y_val, y_test = train_test_valid(remove_punctuation=False)

ds_train = DatasetTransformer(X_train, y_train, tokenizer)
ds_val = DatasetTransformer(X_val, y_val, tokenizer)
ds_test = DatasetTransformer(X_test, y_test, tokenizer)

BATCH_SIZE = 128

train_dataloader = DataLoader(
    ds_train, sampler=RandomSampler(ds_train), batch_size=BATCH_SIZE, num_workers=4
)

val_dataloader = DataLoader(
    ds_val, sampler=SequentialSampler(ds_val), batch_size=BATCH_SIZE, num_workers=4
)

test_dataloader = DataLoader(
    ds_test, sampler=SequentialSampler(ds_test), batch_size=BATCH_SIZE, num_workers=4
)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [10]:
# Instantiate our model and move it to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerModel().to(device)

# Freeze all the Transformer parameters
for p in model.transformer.parameters():
    p.requires_grad = False
# ... except for the bias terms
trainable_params_transformer = [p for (n, p) in model.transformer.named_parameters() if "bias" in n]
for p in trainable_params_transformer:
    p.requires_grad = True

# We'll train the final layer and the bias terms
trainable_params = list(model.fc1.parameters())
trainable_params.extend(list(trainable_params_transformer))

# Define our loss and optimizer
loss_fn = nn.CrossEntropyLoss()
# We'll use AdamW for the Transformer
optim = torch.optim.AdamW(trainable_params)

no_epochs = 20
best_val_loss = 999

Some weights of the model checkpoint at dumitrescustefan/bert-base-romanian-uncased-v1 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
def train_epoch(model, optim, loss_fn, dataloader, epoch_idx):
    """ Trains the model for one epoch and returns the loss together with a classification report
    """
    
    epoch_loss = 0
    # Put the model in training mode
    model.train()
    preds = []
    gt = []
    
    for idx, batch in enumerate(dataloader):
        # Reset gradients
        optim.zero_grad()
        
        inputs, labels = batch
        # Move data to GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        output = model(inputs.to(device))
        # Calculate the loss and backpropagate
        loss = loss_fn(output, labels)
        loss.backward()
        # Update weights
        optim.step()
        
        epoch_loss += loss.item()
        
        probs = F.softmax(output, dim=-1)
        batch_preds = torch.argmax(probs, dim=1)

        preds.append(batch_preds.cpu().numpy())
        gt.append(labels.cpu().numpy())
    
    # Average the epoch losses
    epoch_loss /= len(dataloader)
    
    preds = np.concatenate(preds)
    gt = np.concatenate(gt)
    
    # Get an epoch classification report
    clf_report = classification_report(gt, preds, output_dict=True)
    
    return epoch_loss, clf_report

def test(model, loss_fn, dataloader):
    """ Computes and returns the loss and classification report for a testing dataset.
    """
    
    test_loss = 0
    # put model in evaluation mode
    model.eval()
    preds = []
    gt = []
    
    # Tell PyTorch that we won't be computing gradients
    with torch.no_grad():
        for idx, batch in enumerate(dataloader):
            inputs, labels = batch

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            output = model(inputs)
            test_loss += loss_fn(output, labels).item()
            
            probs = F.softmax(output, dim=-1)
            batch_preds = torch.argmax(probs, dim=1)
            
            preds.append(batch_preds.cpu().numpy())
            gt.append(labels.cpu().numpy())
            
    test_loss /= len(dataloader)
    preds = np.concatenate(preds)
    gt = np.concatenate(gt)
    
    # Get a classification report
    clf_report = classification_report(gt, preds, output_dict=True)
    clf_report_text = classification_report(gt, preds)
    
    return test_loss, clf_report, clf_report_text

In [ ]:
pbar = tqdm(range(no_epochs))

train_losses = []
val_losses = []

train_accs = []
val_accs = []
torch.use_deterministic_algorithms(False)

for e in pbar:
    train_loss, clf_report_train = train_epoch(model, optim, loss_fn, train_dataloader, e)
    val_loss, clf_report_val, _ = test(model, loss_fn, val_dataloader)
    
    train_acc = clf_report_train['accuracy']*100
    val_acc = clf_report_val['accuracy']*100
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    
    pbar.set_description(
        "Epoch: %s, t_loss: %.3f, v_loss: %.3f, t_acc: %.2f, v_acc: %.2f" % (e+1, train_loss, val_loss, train_acc, val_acc))
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), './best_transformer_model.pt')

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with

In [12]:
def plot_curve(val1, val2, val1_name='train', val2_name='valid', curve_name='loss'):
    """ Plots two curves.
    """
    epochs = range(1, len(val1)+1)
    
    plt.plot(epochs, val1, label=f'{val1_name} {curve_name}')
    plt.plot(epochs, val2, label=f'{val2_name} {curve_name}')
    
    plt.title(curve_name)
    plt.legend()
    
    plt.show()

In [ ]:
plot_curve(train_losses, val_losses)

NameError: ignored

In [ ]:
plot_curve(train_accs, val_accs, curve_name='acc')

In [13]:
device
model.load_state_dict(torch.load('./best_transformer_model.pt'))

<All keys matched successfully>

In [16]:
#nu stiu ce l a palit ca inainte mergea fara asta, l-am adaugat acum de ultim moment pentru generare csv
torch.use_deterministic_algorithms(False)
labels = []
ids = []
i = 0
for text1 in X_test:
    text1 = " ".join(normalize_tokens(tweet_tokenizer.tokenize(text1)))
    text1_tokens = tokenizer.encode(text1, add_special_tokens=True, max_length=64, padding='max_length', return_tensors='pt', truncation=True).to(device)
    probabilities = F.softmax(model(text1_tokens), dim=-1)[0]
    if probabilities[0] == max(probabilities):
        labels.append("reporting")
    if probabilities[1] == max(probabilities):
        labels.append("non-offensive")
    if probabilities[2] == max(probabilities):
        labels.append("offensive")
    if probabilities[3] == max(probabilities):
        labels.append("descriptive")
    if probabilities[4] == max(probabilities):
        labels.append("direct")
    ids.append(i)
    i += 1

d = {"Id": ids, "Label": labels}
sub = pd.DataFrame(data=d)
sub.to_csv("./final.csv", index=False)